In [1]:
# use magic command to auto-reload
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import igraph as ig
import json
from dotenv import load_dotenv

sys.path.insert(0, os.path.join(os.getcwd(), '..'))  # parent folder of src
from src.simulation import run_simulation
from src.save import save_replicas_raw
import dotenv
import os
import asyncio
import aiohttp

# load PARAMS.json
with open("PARAMS.json", "r") as f:
    PARAMS = json.load(f)

PARAMS["time_decay_rate"] =  np.log(2)/2  # halving the weight every 2 time steps. change /2 -> /k for halving every k time steps.
PARAMS["W_agent_success"] = 1
PARAMS["W_personal_weights"] = 1
PARAMS["W_post_success"] = 1
PARAMS["noise_level"] = 0.01



ENV = dotenv.dotenv_values("./.env")
HF_TOKEN = ENV["HF_TOKEN"]
API_URL = ENV["API_URL"]
MODEL = ENV["MODEL"]


print(API_URL)
print(MODEL)

# add HF_TOKEN to PARAMS
PARAMS["HF_TOKEN"] = HF_TOKEN
PARAMS["API_URL"] = API_URL
PARAMS["MODEL"] = MODEL



https://router.huggingface.co/v1/chat/completions
meta-llama/Llama-3.1-8B-Instruct:cerebras


In [2]:
timesteps = 4
num_agents = 36
fill_history = 1


PARAMS["num_agents"] = num_agents
PARAMS["timesteps"] = timesteps
PARAMS["fill_history"] = fill_history

In [3]:
List_of_WEIGHTS, List_of_READ_MATRIX, List_of_LIKES, List_of_POSTS = await run_simulation(PARAMS)

T_current: 1
T_current: 2
T_current: 3
T_current: 4
T_current: 1
T_current: 2
T_current: 3
T_current: 4


In [4]:
out_root = "runs2"
save_replicas_raw(
    out_root,
    List_of_WEIGHTS, List_of_READ_MATRIX, List_of_LIKES, List_of_POSTS,
    PARAMS,
    extra_meta={"note": "raw generation only, no metrics"},
)

In [56]:
import os, json, csv, time, uuid, hashlib
import numpy as np

def _flatten_posts_with_replica(List_of_POSTS):
    """Yield (replica, agent, t, text) across all replicas."""
    for r, POSTS in enumerate(List_of_POSTS):
        for agent_id, row in enumerate(POSTS):
            for t, txt in enumerate(row):
                if not txt:
                    continue
                s = str(txt).strip()
                if s:
                    yield (r, agent_id, t, s)

def _sha1_file(path):
    if not os.path.exists(path):
        return None
    import hashlib
    h = hashlib.sha1()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

def save_multi_results(
    out_dir,
    List_of_WEIGHTS,
    List_of_READ_MATRIX,
    List_of_LIKES,
    List_of_POSTS,
    PARAMS,
    prompts_yaml_path="prompts.yaml",
):
    """
    Saves a multi-replica run:

    - arrays_multi.npz: WEIGHTS, READ_MATRIX, LIKES stacked on axis=0 (replica)
    - posts.csv: long format with columns (replica, agent, t, text)
    - metadata.json: params, shapes, reproducibility info
    """
    os.makedirs(out_dir, exist_ok=True)

    # --- 1) Stack arrays along replica axis ---
    WEIGHTS = np.stack(List_of_WEIGHTS, axis=0)       # (R, N, T, max_deg)
    READ_MATRIX = np.stack(List_of_READ_MATRIX, 0)    # (R, N, T, max_deg)
    LIKES = np.stack(List_of_LIKES, 0)                # (R, N, T)

    np.savez_compressed(
        os.path.join(out_dir, "arrays_multi.npz"),
        WEIGHTS=WEIGHTS,
        READ_MATRIX=READ_MATRIX,
        LIKES=LIKES,
    )

    # --- 2) Posts in tidy CSV with replica column ---
    posts_csv = os.path.join(out_dir, "posts.csv")
    with open(posts_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["replica", "agent", "t", "text"])
        w.writerows(_flatten_posts_with_replica(List_of_POSTS))

    # --- 3) Metadata ---
    meta = {
        "run_id": str(uuid.uuid4()),
        "saved_at": time.strftime("%Y-%m-%d %H:%M:%S"),
        "params": PARAMS,
        "replicas": len(List_of_POSTS),
        "shapes": {
            "WEIGHTS": list(WEIGHTS.shape),
            "READ_MATRIX": list(READ_MATRIX.shape),
            "LIKES": list(LIKES.shape),
        },
        "temperature": PARAMS.get("temperature", 0.0),
        "model": PARAMS.get("MODEL"),
        "api_url": PARAMS.get("API_URL"),
        "prompts_yaml": {
            "path": prompts_yaml_path,
            "sha1": _sha1_file(prompts_yaml_path),
        },
        "files": {
            "arrays": "arrays_multi.npz",
            "posts": "posts.csv",
        },
    }
    with open(os.path.join(out_dir, "metadata.json"), "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

def load_multi_results(out_dir):
    """Reads back what save_multi_results wrote."""
    arrays = np.load(os.path.join(out_dir, "arrays_multi.npz"), allow_pickle=True)
    with open(os.path.join(out_dir, "metadata.json"), "r", encoding="utf-8") as f:
        meta = json.load(f)

    posts = []
    posts_path = os.path.join(out_dir, "posts.csv")
    if os.path.exists(posts_path):
        with open(posts_path, "r", encoding="utf-8") as f:
            r = csv.DictReader(f)
            for row in r:
                posts.append((int(row["replica"]), int(row["agent"]), int(row["t"]), row["text"]))

    return {
        "WEIGHTS": arrays["WEIGHTS"],
        "READ_MATRIX": arrays["READ_MATRIX"],
        "LIKES": arrays["LIKES"],
        "POSTS_long": posts,
        "metadata": meta,
    }


In [57]:
R = len(List_of_POSTS)
print("replicas:", R)
print("WEIGHTS[0].shape:", List_of_WEIGHTS[0].shape)   # (N, T_total, max_deg)
print("READ_MATRIX[0].shape:", List_of_READ_MATRIX[0].shape)  # (N, T_total, max_deg)
print("LIKES[0].shape:", List_of_LIKES[0].shape)       # (N, T_total)
print("example post r0 a0 t0:", List_of_POSTS[0][0][0])

replicas: 2
WEIGHTS[0].shape: (6, 3, 5)
READ_MATRIX[0].shape: (6, 3, 5)
LIKES[0].shape: (6, 3)
example post r0 a0 t0: "Just learned that our local school district is cutting funding for arts and music programs. This is a huge step back for our kids and I'm calling on our school board to reverse this decision!" #SaveTheArts #PrioritizeEducation


In [58]:
import numpy as np

WEIGHTS = np.stack(List_of_WEIGHTS, axis=0)      # (R, N, T, max_deg)
READM   = np.stack(List_of_READ_MATRIX, axis=0)  # (R, N, T, max_deg)
LIKES   = np.stack(List_of_LIKES, axis=0)        # (R, N, T)

In [4]:
manifest = load_manifest("runs/2025-09-08_exp1")
print(manifest["replicas"], "replicas available")

# load first replica
r0 = load_replica("runs/2025-09-08_exp1/r000")

print("Array shapes:")
print("WEIGHTS:", r0["WEIGHTS"].shape)
print("READ_MATRIX:", r0["READ_MATRIX"].shape)
print("LIKES:", r0["LIKES"].shape)

print("\nFirst 5 posts:")
for row in r0["POSTS_long"][:5]:
    print(row)

2 replicas available
Array shapes:
WEIGHTS: (6, 3, 5)
READ_MATRIX: (6, 3, 5)
LIKES: (6, 3)

First 5 posts:
(0, 0, '"Just learned that our local school district is cutting funding for arts and music programs. This is a huge step back for our kids and I\'m calling on our school board to reverse this decision!" #SaveTheArts #PrioritizeEducation')
(0, 1, '"Just had a chat with a fellow parent who\'s concerned about the school board\'s priorities. We agree that we need to stand up for our kids and their education, but we also need to cut wasteful spending! It\'s time to get our priorities straight and make a change in our community" #SensibleSolutions #TaxpayerFirst')
(0, 2, '"It\'s time to hold our school board accountable for their decisions. We need to make sure they\'re putting our kids\' needs first, not advancing a radical agenda. Let\'s demand transparency and action at the next school board meeting!" #ParentsUnite #AccountabilityMatters')
(1, 0, '"Stood in line for 5 hours to cast m

In [6]:
pst = List_of_POSTS[0]
msg0 = pst[0]
for i in len(msg0):
    print(msg0[i])

TypeError: 'int' object is not iterable